In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import numpy as np
import tensorflow as tf
import cv2

Tải lên dữ liệu đã trích rút

In [32]:
path="D:/HCSDLDPT/HCSDLDPT_birds_classification/"
path_bird_data= path+"bird_data.npy"
path_rgb=path+"bird_RGB.npy"
path_hsv=path+"bird_HSV.npy"
path_hog=path+"bird_HOG.npy"
path_hog_hsv=path+"concat_hog_hsv.npy"
path_hog_rgb=path+"concat_hog_rgb.npy"

In [33]:
import numpy as np
# Load the NumPy file
flower_data = np.load(path_bird_data, allow_pickle="True")
data_RGB = np.load(path_rgb,allow_pickle="True")
data_HSV = np.load(path_hsv ,allow_pickle="True")
data_HOG = np.load(path_hog , allow_pickle="True")
data_HOG_HSV=np.load(path_hog_hsv, allow_pickle="True")
data_HOG_RGB=np.load(path_hog_rgb, allow_pickle="True")

Hàm Test bằng KNN


In [34]:
import math

def distance_euclidean(x, y):
    if len(x) != len(y):
        return None  # Invalid input
    squared_distance = 0
    for i in range(len(x)):
        squared_distance += (x[i] - y[i]) ** 2
    return squared_distance ** 0.5 

def distance_cosine_similarity(x, y):
    dot_product = sum([xi * yi for xi, yi in zip(x, y)])
    magnitude_x = math.sqrt(sum([xi**2 for xi in x]))
    magnitude_y = math.sqrt(sum([yi**2 for yi in y]))
    return dot_product / (magnitude_x * magnitude_y)

In [35]:
# Tìm k điểm gần nhất
def knn(X_train, y_train, x_new, k):
    distances = []
    for i in range(len(X_train)):
        d = distance_euclidean(X_train[i], x_new)
        distances.append((d, y_train[i]))
    distances.sort()
    k_nearest = distances[:k]
    # Dự đoán nhãn cho điểm dữ liệu mới
    labels = {}
    for distance in k_nearest:
        label = distance[1]
        if label in labels:
            labels[label] += 1
        else:
            labels[label] = 1
    sorted_labels = sorted(labels.items(), key=lambda x: x[1], reverse=True)
    return sorted_labels[0][0]

RGB

In [36]:
#data_RGB
X_RGB_train=[] ;X_RGB_test =[] ; y_RGB_train=[]; y_RGB_test=[]
count=1
for i in data_RGB:
  count+=1
  if count%3==1:
    X_RGB_test.append(i[1])
    y_RGB_test.append(i[0])
  else:
    X_RGB_train.append(i[1])
    y_RGB_train.append(i[0])

In [37]:
from sklearn.metrics import accuracy_score

y_RGB_pred=[]
for i in X_RGB_test:
  res= knn(X_RGB_train,y_RGB_train,i,7)
  y_RGB_pred.append(res)
print(y_RGB_test)
print(y_RGB_pred)
accuracy = accuracy_score(y_RGB_test, y_RGB_pred)
print(f"Accuracy: {accuracy:.2f}")

[1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 15, 17, 19, 20, 20, 21, 22, 23, 25, 26, 27, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 41, 43, 44, 44, 45, 47, 48, 48, 49, 51, 52, 52, 54, 54, 55, 56, 57, 57]
[26, 39, 20, 49, 5, 0, 5, 9, 39, 56, 35, 2, 12, 3, 15, 39, 6, 19, 20, 20, 22, 22, 23, 34, 33, 33, 33, 18, 58, 33, 49, 26, 12, 3, 48, 39, 33, 12, 3, 8, 28, 8, 26, 48, 56, 5, 55, 57, 54, 41, 10, 28, 39, 57]
Accuracy: 0.22


HSV

In [38]:
#data_HSV
X_HSV_train=[] ;X_HSV_test =[] ; y_HSV_train=[]; y_HSV_test=[]
count=1
for i in data_HSV:
  count+=1
  if count%3==1:
    X_HSV_test.append(i[1])
    y_HSV_test.append(i[0])
  else:
    X_HSV_train.append(i[1])
    y_HSV_train.append(i[0])

In [39]:
from sklearn.metrics import accuracy_score

y_HSV_pred=[]
for i in X_HSV_test:
  res= knn(X_HSV_train,y_HSV_train,i,7)
  y_HSV_pred.append(res)
print(y_HSV_test)
print(y_HSV_pred)
accuracy = accuracy_score(y_HSV_test, y_HSV_pred)
print(f"Accuracy: {accuracy:.2f}")

[1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 15, 17, 19, 20, 20, 21, 22, 23, 25, 26, 27, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 41, 43, 44, 44, 45, 47, 48, 48, 49, 51, 52, 52, 54, 54, 55, 56, 57, 57]
[19, 12, 20, 49, 39, 5, 5, 9, 0, 42, 12, 12, 12, 3, 53, 26, 29, 19, 20, 3, 12, 11, 23, 46, 26, 29, 29, 12, 58, 33, 20, 33, 58, 47, 26, 39, 33, 43, 3, 8, 8, 8, 26, 15, 3, 21, 0, 21, 54, 5, 31, 28, 39, 57]
Accuracy: 0.20


HOG

In [40]:
#data_HOG
X_HOG_train=[] ;X_HOG_test =[] ; y_HOG_train=[]; y_HOG_test=[]
count=1
for i in data_HOG:
  count+=1
  if count%3==1:
    X_HOG_test.append(i[1])
    y_HOG_test.append(i[0])
  else:
    X_HOG_train.append(i[1])
    y_HOG_train.append(i[0])

In [41]:
from sklearn.metrics import accuracy_score

y_HOG_pred=[]
progress=0
for i in X_HOG_test:
  res= knn(X_HOG_train,y_HOG_train,i,7)
  y_HOG_pred.append(res)
  print(progress,end=" ")
  progress+=1


print(y_HOG_test)
print(y_HOG_pred)
accuracy = accuracy_score(y_HOG_test, y_HOG_pred)
print(f"Accuracy: {accuracy:.2f}")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 [1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 15, 17, 19, 20, 20, 21, 22, 23, 25, 26, 27, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 41, 43, 44, 44, 45, 47, 48, 48, 49, 51, 52, 52, 54, 54, 55, 56, 57, 57]
[18, 22, 54, 35, 14, 9, 22, 22, 12, 13, 12, 12, 10, 22, 36, 19, 21, 19, 29, 20, 18, 22, 12, 14, 23, 34, 50, 39, 26, 12, 22, 12, 6, 48, 39, 12, 12, 31, 26, 48, 14, 17, 54, 48, 16, 51, 22, 17, 17, 22, 55, 22, 29, 48]
Accuracy: 0.13


HOG + HSV

In [42]:
#data_HOG_HSV
X_HOG_HSV_train=[] ;X_HOG_HSV_test =[] ; y_HOG_HSV_train=[]; y_HOG_HSV_test=[]
count=1
for i in data_HOG_HSV:
  count+=1
  if count%3==1:
    X_HOG_HSV_test.append(i[1])
    y_HOG_HSV_test.append(i[0])
  else:
    X_HOG_HSV_train.append(i[1])
    y_HOG_HSV_train.append(i[0])

In [43]:
from sklearn.metrics import accuracy_score

y_HOG_HSV_pred=[]
progress=0
for i in X_HOG_HSV_test:
  res= knn(X_HOG_HSV_train,y_HOG_HSV_train,i,7)
  y_HOG_HSV_pred.append(res)
  print(progress,end=" ")
  progress+=1

print()
print(y_HOG_HSV_test)
print(y_HOG_HSV_pred)
accuracy = accuracy_score(y_HOG_HSV_test, y_HOG_HSV_pred)
print(f"Accuracy: {accuracy:.2f}")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 
[1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 15, 17, 19, 20, 20, 21, 22, 23, 25, 26, 27, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 41, 43, 44, 44, 45, 47, 48, 48, 49, 51, 52, 52, 54, 54, 55, 56, 57, 57]
[19, 12, 20, 49, 39, 5, 5, 9, 0, 42, 12, 12, 12, 3, 53, 26, 29, 19, 20, 3, 12, 11, 23, 46, 26, 29, 29, 12, 58, 33, 20, 33, 58, 47, 26, 39, 33, 43, 3, 8, 8, 8, 26, 15, 3, 21, 0, 21, 54, 5, 31, 28, 39, 57]
Accuracy: 0.20


HOG + RGB

In [44]:
#data_HOG_RGB
X_HOG_RGB_train=[] ;X_HOG_RGB_test =[] ; y_HOG_RGB_train=[]; y_HOG_RGB_test=[]
count=1
for i in data_HOG_RGB:
  count+=1
  if count%3==1:
    X_HOG_RGB_test.append(i[1])
    y_HOG_RGB_test.append(i[0])
  else:
    X_HOG_RGB_train.append(i[1])
    y_HOG_RGB_train.append(i[0])

In [45]:
from sklearn.metrics import accuracy_score

y_HOG_RGB_pred=[]
progress=0
for i in X_HOG_RGB_test:
  res= knn(X_HOG_RGB_train,y_HOG_RGB_train,i,7)
  y_HOG_RGB_pred.append(res)
  print(progress,end=" ")
  progress+=1

print()
print(y_HOG_RGB_test)
print(y_HOG_RGB_pred)
accuracy = accuracy_score(y_HOG_RGB_test, y_HOG_RGB_pred)
print(f"Accuracy: {accuracy:.2f}")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 
[1, 2, 3, 4, 5, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 15, 17, 19, 20, 20, 21, 22, 23, 25, 26, 27, 27, 28, 29, 30, 31, 33, 34, 36, 38, 39, 41, 43, 44, 44, 45, 47, 48, 48, 49, 51, 52, 52, 54, 54, 55, 56, 57, 57]
[26, 39, 20, 49, 5, 0, 5, 9, 39, 56, 35, 2, 12, 3, 15, 39, 6, 19, 20, 20, 22, 22, 23, 34, 33, 33, 33, 18, 58, 33, 49, 26, 12, 3, 48, 39, 33, 12, 3, 8, 28, 8, 26, 48, 56, 5, 55, 57, 54, 41, 10, 28, 39, 57]
Accuracy: 0.22
